In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import datetime as dt

from sympy import symbols, Symbol, Eq, nsolve, solve, sqrt
from decimal import Decimal, getcontext

import multiprocessing as mp

from LP_Action_Liquidity_Parameter_Function import Get_Liquidity_Parameter, Get_Price

data_dir = '/Users/zhicong/Dropbox/DEXs'
pool_name = 'WETH-USDT 3000 60'

In [2]:
from flipside import Flipside
flipside = Flipside("023b83a2-e411-4849-a961-3742c87bc9c9", "https://api-v2.flipsidecrypto.xyz")

sql = """
SELECT TOKEN0_DECIMALS, TOKEN1_DECIMALS
FROM ethereum.uniswapv3.ez_pools
""" +\
"WHERE POOL_NAME = '" + pool_name + "'" 

query_result_set = flipside.query(sql)
results = flipside.get_query_results(query_result_set.query_id, page_number=1).records[0]
decimals = results['token0_decimals']-results['token1_decimals']
print(decimals)

12


# LP Action

We import all LP actions and conduct basic cleaning.

In [3]:
lp_action = pd.read_csv(os.path.join(data_dir, "Data", "uniswapv3_lp_actions", pool_name + ".csv"))
lp_action.columns = [x.upper() for x in lp_action.columns.tolist()]

# First, remove all newest observations that does not yet have USD value for token
lp_action = lp_action[lp_action['AMOUNT0_USD'].first_valid_index():]
lp_action = lp_action[lp_action['AMOUNT1_USD'].first_valid_index():]
lp_action.reset_index(drop = True, inplace = True)

# USD amount is also not available during a token's early phase, for these, we interploate and use the earliest available price
earliest_token0_price = lp_action.loc[(~lp_action['AMOUNT0_USD'].isna()) & (lp_action['AMOUNT0_ADJUSTED'] != 0),['AMOUNT0_ADJUSTED','AMOUNT0_USD']].tail(1).iloc[0]
earliest_token0_price = earliest_token0_price['AMOUNT0_USD']/earliest_token0_price['AMOUNT0_ADJUSTED']
lp_action.loc[lp_action['AMOUNT0_USD'].isna(),'AMOUNT0_USD'] = lp_action.loc[lp_action['AMOUNT0_USD'].isna(),'AMOUNT0_ADJUSTED']*earliest_token0_price
earliest_token1_price = lp_action.loc[(~lp_action['AMOUNT1_USD'].isna()) & (lp_action['AMOUNT1_ADJUSTED'] != 0),['AMOUNT1_ADJUSTED','AMOUNT1_USD']].tail(1).iloc[0]
earliest_token1_price = earliest_token1_price['AMOUNT1_USD']/earliest_token1_price['AMOUNT1_ADJUSTED']
lp_action.loc[lp_action['AMOUNT1_USD'].isna(),'AMOUNT1_USD'] = lp_action.loc[lp_action['AMOUNT1_USD'].isna(),'AMOUNT1_ADJUSTED']*earliest_token1_price

# Final cleaning
lp_action['BLOCK_NUMBER'] = lp_action['BLOCK_NUMBER'].astype(int)
lp_action.sort_values(by = ['BLOCK_NUMBER'], ascending = False, inplace = True)
lp_action['AMOUNT0_ADJUSTED'] = lp_action['AMOUNT0_ADJUSTED'].astype(float)
lp_action['AMOUNT1_ADJUSTED'] = lp_action['AMOUNT1_ADJUSTED'].astype(float)
lp_action['BLOCK_TIMESTAMP'] = pd.to_datetime([x[:-5] for x in lp_action['BLOCK_TIMESTAMP'].astype(str)])
lp_action = lp_action[(lp_action['AMOUNT0_ADJUSTED'] != 0) | (lp_action['AMOUNT1_ADJUSTED'] != 0)]
lp_action['NF_TOKEN_ID'] = lp_action['NF_TOKEN_ID'].apply(str).str.replace('.0','', regex = False)

# Changing signs of addition and extraction
lp_action['ACTION'] = lp_action['ACTION'].replace({'INCREASE_LIQUIDITY':'Increase Liquidity',
                                                   'DECREASE_LIQUIDITY':'Decrease Liquidity'})
lp_action.rename(columns = {'LIQUIDITY_PROVIDER':'SENDER', 'ACTION':'EVENT_NAME'}, inplace = True)

# Get lower and upper price from tick
lp_action['PRICE_LOWER'] = lp_action['TICK_UPPER'].apply(Get_Price, args = (decimals,))
lp_action['PRICE_UPPER'] = lp_action['TICK_LOWER'].apply(Get_Price, args = (decimals,))

# Reverse to make sure LPs are sorted in the same order as they are executed
lp_action = lp_action.iloc[::-1]
lp_action.reset_index(drop = True, inplace = True)

print("Number of LP Actions in this pool:",len(lp_action))
lp_action.head()

Number of LP Actions in this pool: 36201


,POOL_NAME,TX_HASH,EVENT_NAME,BLOCK_NUMBER,BLOCK_TIMESTAMP,SENDER,NF_TOKEN_ID,AMOUNT0_ADJUSTED,TOKEN0_SYMBOL,AMOUNT0_USD,AMOUNT1_ADJUSTED,TOKEN1_SYMBOL,AMOUNT1_USD,TICK_LOWER,TICK_UPPER,PRICE_LOWER,PRICE_UPPER
0,WETH-USDT 3000 60,0x2e07c690f149223e4f290986277304ea6a05c6ee47ba...,Increase Liquidity,12375326,2021-05-05 16:37:08,0xe14722b5f1aa6ea161e547c5ca598068ec1b3bc0,128,1.242324e+01,WETH,41602.016219,50500.280094,USDT,50501.491412,-197400,-193320,0.00024851989593805513334770496136627571104327...,0.00037371837578048104806793805465758850914426...
1,WETH-USDT 3000 60,0x85a3f0ed4fc14085609e95e7366364566b89d7569e62...,Increase Liquidity,12375529,2021-05-05 17:20:08,0x726022a9fe1322fa9590fb244b8164936bb00489,130,1.000000e-03,WETH,3.410548,0.000000,USDT,0.000000,-195120,-195060,0.00029574991251426623354789380293539124977542...,0.00029752965689780312436713005652677566104102...
2,WETH-USDT 3000 60,0x0fde2929c1fa5dd5dc9a0fcfbf8adccdeefbc398aa36...,Increase Liquidity,12375553,2021-05-05 17:26:26,0x726022a9fe1322fa9590fb244b8164936bb00489,131,9.995372e-08,WETH,0.000341,0.000274,USDT,0.000274,-195180,-195120,0.00029752965689780312436713005652677566104102...,0.00029932011131010653184544856308946236822521...
3,WETH-USDT 3000 60,0x34ca11f7f1c6a4d8d17cabf14c6d0ffa5116a693508c...,Increase Liquidity,12375566,2021-05-05 17:29:37,0x726022a9fe1322fa9590fb244b8164936bb00489,132,9.999750e-06,WETH,0.034105,0.027412,USDT,0.027413,-195180,-195120,0.00029752965689780312436713005652677566104102...,0.00029932011131010653184544856308946236822521...
4,WETH-USDT 3000 60,0x9ece74425a9b5b937b51e14d4fbb9eb7aa6f6ae415d5...,Increase Liquidity,12375566,2021-05-05 17:29:37,0x726022a9fe1322fa9590fb244b8164936bb00489,133,9.999750e-06,WETH,0.034105,0.027412,USDT,0.027413,-195180,-195120,0.00029752965689780312436713005652677566104102...,0.00029932011131010653184544856308946236822521...


# Match Withdrawal with Deposition

In [4]:
"""
token_transfer = pd.read_csv(os.path.join(data_dir, "Data", "uniswapv3_NFT", pool_name + ".csv"))
token_transfer.columns = [x.upper() for x in token_transfer.columns.tolist()]
token_transfer = token_transfer.sort_values(by = ['TOKENID','BLOCK_NUMBER'])

token_transfer = token_transfer[token_transfer['BLOCK_NUMBER'] <= lp_action['BLOCK_NUMBER'].max()]

token_transfer.head()

initial = token_transfer.loc[token_transfer['NFT_FROM_ADDRESS'] == '0x0000000000000000000000000000000000000000',['TOKENID','TX_HASH','NFT_TO_ADDRESS']]
initial['EVENT_NAME'] = 'Increase Liquidity'

final = token_transfer.groupby(['TOKENID'])[['TOKENID','TX_HASH','NFT_TO_ADDRESS']].tail(1)
final['EVENT_NAME'] = 'Decrease Liquidity'

token_transfer = pd.concat([initial,final], axis = 0)
token_transfer.sort_values(by = ['TOKENID'], inplace = True)
token_transfer.rename(columns = {'NFT_TO_ADDRESS':'SENDER'}, inplace = True)

token_transfer.head()
"""

'\ntoken_transfer = pd.read_csv(os.path.join(data_dir, "uniswapv3_NFT", pool_name + ".csv"))\ntoken_transfer.columns = [x.upper() for x in token_transfer.columns.tolist()]\ntoken_transfer = token_transfer.sort_values(by = [\'TOKENID\',\'BLOCK_NUMBER\'])\n\ntoken_transfer = token_transfer[token_transfer[\'BLOCK_NUMBER\'] <= lp_action[\'BLOCK_NUMBER\'].max()]\n\ntoken_transfer.head()\n\ninitial = token_transfer.loc[token_transfer[\'NFT_FROM_ADDRESS\'] == \'0x0000000000000000000000000000000000000000\',[\'TOKENID\',\'TX_HASH\',\'NFT_TO_ADDRESS\']]\ninitial[\'EVENT_NAME\'] = \'Increase Liquidity\'\n\nfinal = token_transfer.groupby([\'TOKENID\'])[[\'TOKENID\',\'TX_HASH\',\'NFT_TO_ADDRESS\']].tail(1)\nfinal[\'EVENT_NAME\'] = \'Decrease Liquidity\'\n\ntoken_transfer = pd.concat([initial,final], axis = 0)\ntoken_transfer.sort_values(by = [\'TOKENID\'], inplace = True)\ntoken_transfer.rename(columns = {\'NFT_TO_ADDRESS\':\'SENDER\'}, inplace = True)\n\ntoken_transfer.head()\n'

Merge token_transfer into lp_action.

# Add Liquidity Parameter

In [5]:
with mp.Pool(10) as pool:
    result = pool.imap(Get_Liquidity_Parameter, lp_action[['AMOUNT1_ADJUSTED','AMOUNT0_ADJUSTED','PRICE_LOWER','PRICE_UPPER']].iterrows(), chunksize = 1)
    output = [x for x in result]

lp_action['L'] = output

Working on row: 0
Working on row: 10000
Working on row: 20000
Working on row: 30000


In [6]:
lp_action.to_csv(os.path.join(data_dir, "uniswapv3_lp_action_LP", pool_name + "_LP.csv"), index=False)